In [1]:
import os
from pathlib import Path
import pandas as pd
import requests

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

import time

In [2]:
url = "https://www.veganrecipeclub.org.uk/recipes/"
SCROLL_PAUSE_TIME = 3

## Load all urls to recipes from the website

In [4]:
def get_html(url):  
    #get chrome driver path
    driver_path = os.path.join(parent(os.getcwd()), 'chromedriver', 'chromedriver.exe.')
    #create instance of Chrome driver
    driver = Chrome(executable_path=driver_path)
    driver.get(url)

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    #load page source
    html = driver.page_source
    #close driver
    driver.close()
    return html

In [5]:
def get_all_urls(html):
    urls = []
    for link in BeautifulSoup(html, parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            urls.append(link['href'])
    return urls

def get_recipes_urls(urls):
    #remove duplicate
    urls = list(set(urls))
    
    #remove url that are not recipes
    for url in urls:
        if(url.startswith("https://www.veganrecipeclub.org.uk/recipes/") == False):
           urls.remove(url)
    return urls

In [6]:
html = get_html(url)
all_urls = get_all_urls(html)
recipes_urls = get_recipes_urls(all_urls)

In [7]:
len(recipes_urls)

696

As of the 4th of october 2021 we have 691 recipes

In [10]:
#optional: save the urls to a csv file
DATA_PATH = Path.cwd().parent.parent / "data" / "raw" / "urls.csv"
urls = pd.DataFrame(recipes_urls, columns=['url'])
urls.to_csv(DATA_PATH)